# Unit network (DEPRECATED - edit unitexpander.py instead)

Give a list of conversion factors between a limited set of commensurable units, find all the possible conversion factors that the list implies.

In [2]:
# %load_ext autoreload
# # provide cell timings
# %load_ext autotime
# %autoreload 2
import sympy as sp
import numpy as np
import pandas as pd
# from sympy import poly, lambdify
from IPython.display import display, Math
# %config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
from itertools import product
from matplotlib import pyplot as plt
%matplotlib widget
from pyperclip import copy as pypercopy
import os, pickle

In [635]:
import networkx as nx
import sympy as sp
from matplotlib import pyplot as plt

In [519]:
%%timeit -n 4
conversions = pd.read_excel("/Users/juan/Google Drive/Zia Lab/Codebase/qdef/data/ConversionFactors.xlsx",None)
conversion_facts = {}
for conversion_type in conversions:
    these_factors = conversions[conversion_type]
    these_dict = dict(zip(list(map(tuple,these_factors[['Source','Destination']].values.tolist())),
          list(map(lambda x: x[0],these_factors[['Factor']].values.tolist()))))
    conversion_facts.update(these_dict)

8.83 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 4 loops each)


In [715]:
conversions = pd.read_excel(os.path.join(module_dir,'data','ConversionFactors.xlsx'),None)
conversion_facts = {}
for conversion_type in conversions:
    these_factors = conversions[conversion_type]
    these_dict = dict(zip(list(map(tuple,these_factors[['Source','Destination']].values.tolist())),
          list(map(lambda x: x[0],these_factors[['Factor']].values.tolist()))))
    conversion_facts.update(these_dict)
cfacts = conversion_facts
unitgraph = nx.DiGraph()
allunits = set()
for k,v in cfacts.items():
    unitgraph.add_node(k[0])
    unitgraph.add_node(k[1])
    v = sp.S(v)
    unitgraph.add_weighted_edges_from([(k[0],k[1],v)])
    unitgraph.add_weighted_edges_from([(k[1],k[0],1/v)])
    allunits.add(k[0])
    allunits.add(k[1])
allunits = list(allunits)
allpairs = list(combinations(allunits,2))
for pair in allpairs:
    try:
        path = nx.shortest_path(unitgraph,*pair)
    except:
        path = []
    if len(path) == 0:
        continue
    fromto = (path[0],path[-1])
    backto = (path[-1], path[0])
    total_factor = sp.S(1)
    for node_idx in range(len(path)-1):
        nodeA = path[node_idx]
        nodeB = path[node_idx+1]
        total_factor *= unitgraph[nodeA][nodeB]['weight']
    if fromto not in cfacts.keys():
        cfacts[fromto] = float(total_factor)
    if backto not in cfacts.keys():
        cfacts[backto] = float(1/total_factor)
for k,v in cfacts.items():
    unitgraph.add_node(k[0])
    unitgraph.add_node(k[1])
    v = sp.S(v)
    unitgraph.add_weighted_edges_from([(k[0],k[1],v)])
    unitgraph.add_weighted_edges_from([(k[1],k[0],1/v)])

In [725]:
pickle.dump(cfacts,open(os.path.join(module_dir,'data','conversion_facts.pkl'),'wb'))

In [721]:
# Script version
# from itertools import combinations
# import pandas as pd
# import networkx as nx
# import os
# import pickle

# def main():
#     if os.path.exists('./conversion_facts.pkl'):
#         print("conversion_facts.pkl already exists, delete if you want to generate it again.")
#         return None
#     conversions = pd.read_excel(os.path.join(module_dir,'data','ConversionFactors.xlsx'),None)
#     conversion_facts = {}
#     for conversion_type in conversions:
#         these_factors = conversions[conversion_type]
#         these_dict = dict(zip(list(map(tuple,these_factors[['Source','Destination']].values.tolist())),
#               list(map(lambda x: x[0],these_factors[['Factor']].values.tolist()))))
#         conversion_facts.update(these_dict)
#     cfacts = conversion_facts
#     unitgraph = nx.DiGraph()
#     allunits = set()
#     for k,v in cfacts.items():
#         unitgraph.add_node(k[0])
#         unitgraph.add_node(k[1])
#         v = sp.S(v)
#         unitgraph.add_weighted_edges_from([(k[0],k[1],v)])
#         unitgraph.add_weighted_edges_from([(k[1],k[0],1/v)])
#         allunits.add(k[0])
#         allunits.add(k[1])
#     allunits = list(allunits)
#     allpairs = list(combinations(allunits,2))
#     for pair in allpairs:
#         try:
#             path = nx.shortest_path(unitgraph,*pair)
#         except:
#             path = []
#         if len(path) == 0:
#             continue
#         fromto = (path[0],path[-1])
#         backto = (path[-1], path[0])
#         total_factor = sp.S(1)
#         for node_idx in range(len(path)-1):
#             nodeA = path[node_idx]
#             nodeB = path[node_idx+1]
#             total_factor *= unitgraph[nodeA][nodeB]['weight']
#         if fromto not in cfacts.keys():
#             cfacts[fromto] = float(total_factor)
#         if backto not in cfacts.keys():
#             cfacts[backto] = float(1/total_factor)
#     for k,v in cfacts.items():
#         unitgraph.add_node(k[0])
#         unitgraph.add_node(k[1])
#         v = sp.S(v)
#         unitgraph.add_weighted_edges_from([(k[0],k[1],v)])
#         unitgraph.add_weighted_edges_from([(k[1],k[0],1/v)])
#     pickle.dump(cfacts,open('./conversion_facts.pkl','wb'))
#     return None
# if __name__ == '__main__':
#     main()

In [739]:
plt.figure()
nx.draw(unitgraph)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …